# Experimentation 3: Seizure Detection

## Structure
1. Data preparation (pre-processing)<br/>
   Note: ideally you store the pre-processed data in template 02 and load it here again, instead of duplicating the code …
2. Windowing
3. Feature & label extraction
4. Data split
5. FV-pre-processing
6. Classifier creation
7. Evaluation


## Reporting
Not every plot or number generated in this template comes into the report.
Nonetheless, some intermediary steps are required to achieve the relevant results.
Cells, which have a *direct* relation to a part of the report, 
are marked with **Report:**.
Be sure to not jump over relevant precursor steps.
But when having little time, think about your priorities, and try to avoid spending time on decorative steps or cells which are not needed for the report.

The relevant information about what to include in the report is the corresponding pdf document on itslearning.
Annotations in this file are just for orientation and might not be complete.



## What to do
  - Build it up step by step, i.e
      * First take a short look at how the pipeline looks like
      * Initially, use the raw window as feature vector for starters to get everything going
      * Implement the overlap of the window with the seizure,
        so that you have a label
      * Only then start implementing features
      * Then, you can go on
  - There are many parameters which influence the outcome
      * Which influence does balancing have?
      * How about scaling?
      * And PCA with / without whitening?
      * Try to get a feeling for how the PCA changes the channels, 
        i.e., plot consecutive channels on x/y axis.
        Colour the dots according to their labels.
      * How do your observations relate to your expectations?
  - Investigate the relevance of features
  - Without doing a proper evaluation, do you think that the relevance of the features correlates with the computaton time going into deriving them?
  
## Discussion
In your group or with your neighbours
- Which influence does the data preparation have? I.e., normalisation, PCA, whitening?
- When comparing the use of PCA here to Module 2, how would you describe the interpretability of the dimensions?
- Which performance do you achieve? Is the detector usable?
- Reflecting on the performance of the detector, for which tasks would you trust this model? How would you like to change the performance for possible use cases?

## When you are done with all other tasks…
You can check template 3a for optional tasks to dig deeper.

In [ ]:
from pyedflib import highlevel
from pathlib import Path

import numpy as np


# Jupyter lab supports interactive plots      # Matplotlib for plotting
# using "widget"
%matplotlib widget

# Jupyter lab doesn't support notebook,
# which was the preferred method for jupyter notebooks.
#%matplotlib notebook
#%matplotlib inline


from matplotlib import pyplot as plt
from matplotlib import patches

# Adjust plot size & resolution for inline display.
# Tune to your needs.
plt.rcParams['figure.figsize'] = [9, 5.56]
plt.rcParams['figure.dpi'] = 100

In [ ]:
# Defining base paths for read-only and read-write data
# will make it easy for us to switch between cloud
# and local environments by just adjusting the paths.
#
# Also, it will prevent accidental overwriting of read-only data.
#
# The example codes starting with '/work' relate to UCloud.
# Note that jupyterlab in ucloud will not show you the /work folder.
# What jupyterlab shows as origin */* of the filesystem
# is in reality the */work* folder.

from pathlib import Path         # OS agnostic path handling (/ vs \)

user = 'jb'                      # Per-user output directories

# Base directories
# DATA_DIR -- where the read-only sources are
DATA_DIR = Path('/work/data')

# OUTPUT_DIR -- where we will keep our data (read/write)
# We will make sure it exists!
OUTPUT_DIR = Path('/work/output')

# Now create our own output directory and change to it
OUTPUT_DIR /= user
OUTPUT_DIR.mkdir(parents=True, exist_ok=True)

import os
os.chdir(OUTPUT_DIR)             # Change working directory

# Loading the Data
While this template replicates the boxes from template 02, it is advised to store the pre-processed data in template 02 and then load it again here, preventing you from duplicating all the code.

In [ ]:
# Loading file:
fn = DATA_DIR / \
    'module_3_-_time_series_analysis_for_eeg' / \
    'seizure_eegs/' \
    '01.edf'

print('Reading EEG from: {}'.format(fn))

raw_signals, signal_headers, header = \
    highlevel.read_edf(str(fn))

In [ ]:
# Annotations:
# In seconds since start of recording,
# use singal_headers.sample_frequency to map to sample numbers
seizures = list()
for start, duration, kind in header['annotations']:
    if kind == 'Seizure':
        seizures.append((start, float(duration)))

In [ ]:
# For easier referencing of a specific channel:
# A lookup table with the labels of all channels in the edf
for i, sh in enumerate(signal_headers):
    print('Channel {:2d}: {}'.format(i, sh['label']))

# Pre-Processing & Preparation

## Filtering
Take from Task 2, or load pre-processed data from file.
Note that you need to filter all channels.

In [ ]:
# TODO: REPLACE the following line of code!
#       There is no need to keep it.
signals = raw_signals.copy()

## Per-channel normalisation
The signals have per definition a zero line.
The easiest way to go on is therefore to scale symmetrically.
Note though that this is a plump assumption and should be replaced by
transforming to physical units first.

Why do we normalise here?
Well, we don't have to, but it is especially helpful for methods includig probability density function (pdf) estimation (Optional Tasks 3a).

In [ ]:
# We need to pass the values on to a new variable,
# even if we by default do not filter the normalise
# the signal here.
norm_signals = None

# Change to True to perform normalisation here.
if False:
    norm_signals = np.zeros((signals.shape[0] - 1, signals.shape[1]))
    for i in range(signals.shape[0] - 1):
        # Using variance should not be that sensitive to outliers as the maximum.
        # But both perform good. For pdf estimation use max of absolute value.
        #scaling_factor = 2 * np.sqrt(np.var(signals[i,:]))
        scaling_factor = np.max(np.abs(signals[i,:]))
        if scaling_factor == 0:
            print('Channel #{} has no content!'.format(i))
        norm_signals[i,:] = signals[i,:] / scaling_factor
else:
    norm_signals = signals.copy()

## Windowing function & overlap with seizure
There are library functions which also do this.
This implementaion is rather raw, but illustrates the idea.

We analyse the time series in terms of windows. Given a signal + header,
we can configure the window length + overlap.

In [ ]:
from scipy.signal import windows
    
def interval_overlap(i1_start : float, i1_stop : float,
            i2_start : float, i2_stop : float) -> float: 
    
    """ A function to determine the overlap between intervals.

        There are two options to implement this function.

        (1) For starters, determine if there is an overlap.
            If there is, then return 1

        (2) Determine the actual overlap as a fraction in [0, 1],
            return this value.
    
            Given the intervals [i1_start, i1_stop] and [i2_start, i2_stop],
            return the overlap normalised to the smaller interval's length.
    
            This overlap will then be used determine how much of an interval
            is seizure.
    """

    # TODO: Implement the overlap between two intervals
    pass


def overlap_with_seizures(i1_start : float, i1_duration : float) -> float:
    """ Determine the overlap of i1 with annotated seizures. """

    result : float = 0
    for _start, _duration in seizures:
        this = interval_overlap(i1_start, i1_start + i1_duration, _start, _start + _duration)
        result += this
        
    return result


def time_windows(signal, header, window_length_in_s, overlap_in_s):
    """ Iterator over all time windows of a certain length.
    
        For a given signal, the return value of this function
        can be iterated over to access all windows of specified
        length and overlap.
    """
    rate = header['sample_frequency']
    window_length_in_samples = int(np.trunc(window_length_in_s * rate))
    overlap_in_samples = int(np.trunc(overlap_in_s * rate))
    step = window_length_in_samples - overlap_in_samples
    
    for i in range(0, len(signal), step):
        if (len(signal) - i) < window_length_in_samples:
            print('Information: incomplete window encountered. This is normal for the last window of a channel.')
            return

        yield (i + window_length_in_samples/2) / rate, \
              signal[i:i + window_length_in_samples], \
              overlap_with_seizures(i / rate, window_length_in_samples / rate)

# Feature extraction
For each window in each channel, we will generate a feature vector.
The label will be determined by the overlap with the seizure annotation.

General procedure:
 1. Per-channel normalisation
 2. Window splitting (time, signal, overlap of window with seizure annotation)
 3. Calculation of features
 4. Store: (time, features, label == overlap)
 
I suggest to implement at least the following features
  * mean
  * variance
  * energy
  * area
  * nonlinear_energy
  * num_zero_crossings
  * length_of_curve

Please check with the lecture to get a detailed list of what is expected for a minimal implementation.

**Report:** Part 3.3

## Code for Calculating Features
Of course, with more complex features, you are free to map this in the notebook structure.

In [ ]:
def feature1(signal):
    """ Calculate feature 1 on signal. """
    pass


def feature2(signal):
    """ Calculate feature 2 on signal. """
    pass

## Forming the Feature Vector
Here, you actually plug in the features you want to include in the feature vector.

In [ ]:
def build_feature_vector(sig):
    """ Create a feature vector by combining all features. """
    return np.array([
        feature1(sig),
        feature2(sig),
    ])

# We derive the number of features automatically
# by calling build_feature_vector with fake data.
num_features = len(build_feature_vector(np.linspace(0, 19, 20)))
print('Number of features per fv:', num_features)

## Processing the Data
Now perform the feature calculations and collect the feature vectors for all windows.

**Report:** Part 3.3

In [ ]:
# Per channel window splitting & feature vector creation
# Optimisation options:
#   * Save all windows, so that we can calculate features without window regeneration!
#   * Extract windows for all channels at the same time
window_length  = 3 # s
window_overlap = 1 # s

# For calculation of the number of windows
window_length_in_samples = int(np.trunc(window_length * signal_headers[0]['sample_frequency']))
overlap_in_samples = int(np.trunc(window_overlap * signal_headers[0]['sample_frequency']))

num_channels = norm_signals.shape[0]
num_windows = int(np.trunc(
    (norm_signals.shape[1] - overlap_in_samples) / (window_length_in_samples - overlap_in_samples)
))

fv_times = np.zeros((num_windows))
fvs = np.zeros((num_windows, num_channels * num_features))
labels = np.zeros((num_windows))

_window = None
for ch_idx in range(norm_signals.shape[0]):
    print('Extracting features for Channel #' + str(ch_idx), 'of', norm_signals.shape[0])
    for win_idx, (start_time, win_data, overlap) in enumerate(time_windows(norm_signals[ch_idx], signal_headers[ch_idx], 
                                                                           window_length, window_overlap)):
        fvs[win_idx, ch_idx * num_features:(ch_idx+1) * num_features] = build_feature_vector(win_data)
        if (ch_idx == 0):
            fv_times[win_idx] = start_time
            labels[win_idx] = overlap
        if (ch_idx == 0) and (win_idx == 0):
            _window = win_data

print(num_windows)
print(num_channels, num_features, num_channels * num_features)
print(fvs.shape)

# Classification
Now comes the time to throw the feature vectors at different classifiers.
But as we do not have the time to go into all the different options, we will stick to the logistic regression, which we know from Module 2 as a possible choice for classification tasks.

**Report:** Part 3.3

## Data Set Splitting
While there are plenty of methods available to automate this part, we are here going to do it by hand, so that we can experiment with the process, i.e., you definitely want to fiddle with the balancing.

**Report:** Part 3.3

In [ ]:
# Preparation of data sets for training and evaluation
balanced = False
test_split = 0.8
non_seizure_overhead = 1

binary_labels : np.ndarray = labels > 0

train_data : np.ndarray = None
train_labels : np.ndarray = None
test_data : np.ndarray = None
test_labels : np.ndarray = None

if balanced:      # Select seizures (lower number), and select interictal accordingly
    from numpy.random import default_rng
    rng = default_rng()

    num_seizure_fvs = sum(binary_labels)
    for_training = int(np.round(test_split * num_seizure_fvs))
    print('Number of fvs with seizure:', num_seizure_fvs)
    print('  Number of seizure training fvs:', for_training)
    print('  Number of seizure validation fvs:', num_seizure_fvs - for_training)
    print('  Number of interictal validation fvs:', 
          len(binary_labels) - num_seizure_fvs - non_seizure_overhead * for_training)
    
    # Get indices of ictal and interictal fvs.
    # Should we shuffle them to make sure that we get a somewhat
    # random selection? Might be bad, because this way, especially
    # with the low numbers, we could end up only seeing beginnings…
    seizure_idx = np.nonzero(binary_labels >= 0.5)[0]
    interictal_idx = np.nonzero(binary_labels < 0.5)[0]
    
    # The classifier fit(…) will shuffle. But just in case we do so too.
    training_idx = np.concatenate(
        (seizure_idx[:for_training], interictal_idx[:non_seizure_overhead * for_training])
    )
    np.random.shuffle(training_idx)
    test_idx = np.concatenate(
        (seizure_idx[for_training:], interictal_idx[non_seizure_overhead * for_training:])
    )
    np.random.shuffle(test_idx)

    _train_data = fvs[training_idx,:]
    train_labels = binary_labels[training_idx]
    _test_data = fvs[test_idx,:]
    test_labels = binary_labels[test_idx]
    
else:             # Just split according to test_split
    last = int(test_split * len(binary_labels))

    _train_data = fvs[:last,:]
    train_labels = binary_labels[:last]
    _test_data = fvs[last:,:]
    test_labels = binary_labels[last:]

print('\nData summary')
print('  Number of feature vectors:', fvs.shape[0])
print('  Size of training set:', _train_data.shape[0])
print('  Size of validation set:', _test_data.shape[0])

## Last Pre-Processing & PCA
In this part, we investigate the improvements by using scaling and PCA for reducig dimensions and decorrelating signals.
Please check [the scikit documentation on pre-processing](https://scikit-learn.org/stable/modules/preprocessing.html) for details on how the following settings are transforming the data.

**Report:** Part 3.3

In [ ]:
# Data pre-processing
# https://scikit-learn.org/stable/modules/preprocessing.html
#
# Scaling and PCA
# Also check the order of PCA with/without whitening and scaling.

do_PCA = True
PCA_whiten = True
PCA_n_comp = 'mle'

from sklearn import preprocessing

class DummyScaler():
    def fit(self, data):
        return self
    
    def transform(self, data):
        return data


scaler = DummyScaler().fit(_train_data)
#scaler = preprocessing.StandardScaler().fit(_train_data)
#scaler = preprocessing.RobustScaler().fit(_train_data)

train_data = scaler.transform(_train_data)
test_data = scaler.transform(_test_data)

print('Original dimensionality:', _train_data.shape[1])

if do_PCA:
    # Use PCA to reduce linear dependencies
    from sklearn.decomposition import PCA

    pca = PCA(n_components=PCA_n_comp, svd_solver='full', whiten=PCA_whiten, copy=True)
    pca.fit(train_data)
    train_data = pca.transform(train_data)
    test_data = pca.transform(test_data)

    print('PCA performed. # reduced dimensions:', pca.n_components_)
    print('   Ratio of variance explained:', pca.explained_variance_ratio_[:4])

## Optional: Inspecting the PCA's Effect
For trying to get a feeling for how the PCA changes the channels, please plot consecutive or two otherwise selected channels on x/y axis.
Colour the dots according to their labels.

Can you find dimensions which separate the labels better than others?
Keep them in mind when checking later the relevance of the features.

This part is for trying to get a feeling of how the PCA might help the classifier,
it is an explorative task, which means trying out, plotting, and inspecting.
Do this only after you have completed your report, to avoid getting stuck here.

# Create a Model
**Report:** Part 3.3

In [ ]:
from sklearn.linear_model import LogisticRegression

classifier = LogisticRegression()

In [ ]:
# Fit the model
classifier.fit(train_data, train_labels)

# Evaluation
Evaluate accuracy, sensitivity, specificity, confusion matrix.
What do these measures tell us about the performance of our model?

Check which features were most relevant.

**Report:** Part 3.3